In [1]:
! pip install transformers

     |████████████████████████████████| 1.9MB 9.1MB/s 
     |████████████████████████████████| 3.2MB 34.5MB/s 
     |████████████████████████████████| 890kB 34.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=8d7f3407fea3f7bdd627934d0c9a9c217d2542f9cf605a8fc149b8860fe36357
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Using Cloud Storage

Authenticate with Google Account

In [2]:
from google.colab import auth
auth.authenticate_user()

Install GCloud SDK

In [ ]:
!curl https://sdk.cloud.google.com | bash

Initialize SDK - need to answer some questions

In [ ]:
!gcloud init

Download files

* !gsutil cp URI .  You can replace the . with the name you want
* To get URI go to file you want to dowload in Google Cloud Platform and copy it 

In [5]:
!gsutil cp gs://capestone-benchmark-data/apm/metadata/0002cf94-5d45-48ef-9ca9-14c7e85ecef1.json metadata.json
!gsutil cp gs://capestone-benchmark-data/apm/box/0002cf94-5d45-48ef-9ca9-14c7e85ecef1.csv annotation.csv
!gsutil cp gs://capestone-benchmark-data/apm/img/0002cf94-5d45-48ef-9ca9-14c7e85ecef1.png image.png

Copying gs://capestone-benchmark-data/apm/metadata/0002cf94-5d45-48ef-9ca9-14c7e85ecef1.json...
/ [1 files][  227.0 B/  227.0 B]                                                
Operation completed over 1 objects/227.0 B.                                      
Copying gs://capestone-benchmark-data/apm/box/0002cf94-5d45-48ef-9ca9-14c7e85ecef1.csv...
/ [1 files][ 18.2 KiB/ 18.2 KiB]                                                
Operation completed over 1 objects/18.2 KiB.                                     
Copying gs://capestone-benchmark-data/apm/img/0002cf94-5d45-48ef-9ca9-14c7e85ecef1.png...
/ [1 files][  4.6 MiB/  4.6 MiB]                                                
Operation completed over 1 objects/4.6 MiB.                                      


# Model Code

In [6]:
from PIL import Image, ImageDraw, ImageFont

labels = ["Total", "Other"]
label_map = {i: label for i, label in enumerate(labels)}
type(label_map)

dict

In [ ]:
from transformers import LayoutLMModel, LayoutLMConfig, LayoutLMTokenizer

# Define Tokenizer
tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")


In [38]:
#ari's code_box for dataloader
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Define pytorch Dataset

class InvoiceDataset(Dataset):

    def __init__(self, csv_file, metadata):
        self.annotations = pd.read_csv(csv_file, index_col=0).to_numpy()
        self.metadata = metadata
        self.sequence_length = 32
        self.pre_process(self.annotations, self.metadata)

    def pre_process(self, annotations, metadata):
        for index, row in enumerate(self.annotations):
            if row[6] == metadata["AMOUNT"]:
                self.annotations[index][7] = "total"
            else:
                self.annotations[index][7] = "other"

    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        x = self.annotations[idx]
        temp = tokenizer(x[6])
        input_id = temp['input_ids']
        input_id.extend([0] * (self.sequence_length - len(temp['input_ids']))) 
        input_id = torch.tensor([input_id], dtype=torch.long)
        label = torch.tensor([[1] * self.sequence_length]) if x[7] == "total" else torch.tensor([[-1] * self.sequence_length])

        return input_id, label
    def normalize_bbox(bbox, width, height):
        # Width and height correspond to those of the original image
        return [
            int(1000 * (bbox[0] / width)),
            int(1000 * (bbox[1] / height)),
            int(1000 * (bbox[2] / width)),
            int(1000 * (bbox[3] / height)),
        ]



In [ ]:
#copy of dataloader code box - working on it to create InvoiceDataset
import torch 
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

# Define pytorch Dataset

class InvoiceDataset(Dataset):

    def __init__(self, root_dir, transform = None):
        self.annotations = __get_data__(root_dir + '/box')
        self.root_dir = root_dir # Directory with all the images

    def __get_data_one_file__(path):
      #using path to annotations
      invoice = pd.read_csv(path,header=0)
      invoice = invoice[['x0','x2','y0','y2','page_width','page_height','content']]
      invoice_df = []
      for i in invoice:
        invoice_df.extend([normalize_bbox([i['x0'],i['x2'],i['y0'],i['y2']], i['page_width'], i['page_height']), i['content']])
      return invoice_df

    def __get_data__(root_dir_annotations):
      annotation_files = sorted(glob(root_dir_annotations+'/*.csv'))

      all_invoices_annotations = []

      for i in annotation_files:
        invoice_df = get_data_one_file(i)
        all_invoices_annotations = pd.concat([all_invoices, invoice_df], axis=0, ignore_index = True)
        
      return(all_invoices_annotations)

    def __len__(self):
        return (self.annotations).shape[0]
    
    def __getitem__(self, idx):
        return self.annotations[i]


        
def normalize_bbox(bbox, width, height):
     # Width and height correspond to those of the original image
     return [
         int(1000 * (bbox[0] / width)),
         int(1000 * (bbox[1] / height)),
         int(1000 * (bbox[2] / width)),
         int(1000 * (bbox[3] / height)),
     ]

# # idx = 0
# # bbox = normalize_bbox([df.loc[idx,'x0'],df.loc[idx,'x2'],df.loc[idx,'y0'],df.loc[idx,'y2']], df.loc[idx,'page_width'], df.loc[idx,'page_height'])

# path_to_datasets = ...
# data_sets = datasets.DatasetFolder(path_to_datasets, loader=get_data_one_file, extensions=['.csv'])
# train_loader = torch.utils.data.DataLoader(data_sets, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
from transformers import LayoutLMForTokenClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model
model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=len(labels))
model.to(device)

In [29]:
# Try forward pass with a single example
sequence_length = 500
bbox = [160, 301, 58, 113]
content = "TXU"
label = "Other"

# bbox_pad = torch.zeros(4)
# bbox = normalize_bbox([160,301,58,13], 2000, 2600)
bbox=[[[1,2,3,5], [1,2,45,2], [6,5,42,1], [7,52,12,9]]]

bbox = torch.tensor(bbox, dtype=torch.long)
# bbox_pad[:4] = bbox
print("bbox: ",bbox)
temp = tokenizer(content)
print(tokenizer.decode(temp['input_ids']))
print("type input_ids = ", type(temp['input_ids']))
input_ids = torch.tensor([temp['input_ids']], dtype=torch.long)
print('input_ids: ', input_ids)
token_type_ids = torch.tensor([temp['token_type_ids']], dtype=torch.long)
labels = torch.tensor([[1, 0, 0, 0]])
print(labels.shape)
attention_mask = torch.tensor([temp['attention_mask']], dtype=torch.long)
print(input_ids, token_type_ids, labels, attention_mask)

print(bbox.shape, input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
# out = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)

out = model(input_ids = input_ids, labels = labels)
print(out)

bbox:  tensor([[[ 1,  2,  3,  5],
         [ 1,  2, 45,  2],
         [ 6,  5, 42,  1],
         [ 7, 52, 12,  9]]])
[CLS] txu [SEP]
type input_ids =  <class 'list'>
input_ids:  tensor([[  101, 19067,  2226,   102]])
torch.Size([1, 4])
tensor([[  101, 19067,  2226,   102]]) tensor([[0, 0, 0, 0]]) tensor([[1, 0, 0, 0]]) tensor([[1, 1, 1, 1]])
torch.Size([1, 4, 4]) torch.Size([1, 4]) torch.Size([1, 4]) torch.Size([1, 4]) torch.Size([1, 4])
TokenClassifierOutput(loss=tensor(0.5642, grad_fn=<NllLossBackward>), logits=tensor([[[-0.4792,  0.2586],
         [ 0.1359,  0.1439],
         [ 0.0927,  0.0447],
         [-0.4061, -0.8401]]], grad_fn=<AddBackward0>), hidden_states=None, attentions=None)


In [ ]:
# Try forward pass with a example of 32 padding
sequence_length = 32
bbox = [160, 301, 58, 113]
content = "TXU"
label = "Other"
width = 2000
height = 2600

bbox = normalize_bbox(bbox, width, height)
bbox = torch.tensor(bbox, dtype=torch.long)
print(bbox)

temp = tokenizer(content)
print(temp)

In [ ]:

words = ["worthless", "world"]
normalized_word_boxes = [637, 773, 693, 782], [698, 773, 733, 782]
token_boxes = []
for word, box in zip(words, normalized_word_boxes): 
    word_tokens = tokenizer.tokenize(word)
    print(len(word_tokens))
    token_boxes.extend([box] * len(word_tokens))
# add bounding boxes of cls + sep tokens
token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]
encoding = tokenizer(' '.join(words), return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])
outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids)
# last_hidden_states = outputs.last_hidden_state
print(bbox, input_ids.shape, attention_mask, labels.shape)
print(word_tokens)
print(outputs)


1
1
tensor([[[   0,    0,    0,    0],
         [ 637,  773,  693,  782],
         [ 698,  773,  733,  782],
         [1000, 1000, 1000, 1000]]]) torch.Size([1, 4]) tensor([[1, 1, 1, 1]]) torch.Size([1, 4])
['world']
TokenClassifierOutput(loss=None, logits=tensor([[[-0.1785, -0.1037],
         [-0.1754,  0.0262],
         [-0.0732, -0.1393],
         [-0.1804, -0.1038]]], grad_fn=<AddBackward0>), hidden_states=None, attentions=None)


In [ ]:
# Initialize Dataset
invoice_dataset = InvoiceDataset("annotation.csv", {"AMOUNT":52.62})
train_dataloader = DataLoader(invoice_dataset,
                              batch_size=1)

for batch in train_dataloader:
    print(batch[0].squeeze(0).shape, batch[1].squeeze(0).shape, batch[0].squeeze(0))
    model(input_ids = batch[0].squeeze(0), labels = batch[1].squeeze(0))

In [ ]:
# Training

from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 5
t_total = len(train_dataloader) * num_train_epochs # total number of training steps 

#put the model in training mode
model.train()
for epoch in range(num_train_epochs):
    for batch in tqdm(train_dataloader, desc="Training"):
        input_ids = batch[0].to(device)
    #   bbox = batch[4].to(device)
    #   attention_mask = batch[1].to(device)
    #   token_type_ids = batch[2].to(device)
    #   labels = batch[3].to(device)
        labels = batch[1].to(device)

        # forward pass
    #   outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,
    #                   labels=labels)
        # print("\ninput_ids: ", input_ids.squeeze(0).shape)
        # print("labels: ", labels.squeeze(0).shape)
        outputs = model(input_ids = input_ids.squeeze(0), labels = labels.squeeze(0))
        loss = outputs.loss
        print("here!")
        # print loss every 100 steps
        if global_step % 100 == 0:
            print(f"Loss after {global_step} steps: {loss.item()}")

        # backward pass to get the gradients 
        loss.backward()

        #print("Gradients on classification head:")
        #print(model.classifier.weight.grad[6,:].sum())

        # update
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1

# Legacy Code

In [ ]:
coordinates = [50, 80, 70, 100]
sequence = "total"
metadata = {"UID":"0002cf94-5d45-48ef-9ca9-14c7e85ecef1","AMOUNT":52.62,"WO_NUM":None,"IMI_REFERENCE":"100072514116","INVOICE_CHECK_MEMO":"100072514116","TXN_REFERENCE":None,"VHOST":"butlerpropertycompany","CREATED_AT":"2020-11-06 21:48"}
metadata['AMOUNT']

52.62

In [ ]:
from torch.nn.functional import pad

labels = torch.tensor([[1, 0, 0, 0]])
labels
# m = ConstantPad1d(32,0)
# m(labels)

source_pad = pad(labels, pad=(0, 0, 0, 70 - labels.shape[0]))
source_pad

In [ ]:
with open()

In [10]:
import json
import csv
import pandas as pd
with open("metadata.json") as json_file:
    data = json.load(json_file)
with open("annotation.csv") as csv_file:
    rows = {}
    rows['form'] = []
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        continue
        # print(row)
df = pd.read_csv("annotation.csv", index_col=0)

In [13]:
df_arr = df.to_numpy()
df_arr
for index, row in enumerate(df_arr):
    if row[6] == data["AMOUNT"]:
        df_arr[index][7] = "total"
    else:
        df_arr[index][7] = "other"
# df_arr[5]
x = df_arr[1]
sequence_length = 32
temp = tokenizer(x[6])
input_id = temp['input_ids']
input_id.extend([0] * (32 - len(temp['input_ids'])))
print(input_id)
input_id = torch.tensor([input_id], dtype=torch.long)
print(input_id.shape)
# asd = torch.tensor([[1] * sequence_length])
# print(input_id.shape, asd.shape)
# out = model(input_ids = input_id, labels = asd)



[101, 2943, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
torch.Size([1, 32])
